In [6]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np
from perceptron_p3 import Perceptron  # our custom class
from perceptron_p3 import MultiClassPerceptronOvR 
from perceptron_p3 import MultiClassAdaBoostSAMME

# Charger le dataset
digits = load_digits()
X = digits.data
y = digits.target
n_classes = len(np.unique(y))

# Séparer en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Créer et entraîner un SAMME
adb = MultiClassAdaBoostSAMME(base_estimator_class=MultiClassPerceptronOvR,
                               n_classes=n_classes, 
                               M=50,            # par ex
                               alpha=0.1,       # learning rate du perceptron
                               epochs=5)        # nb d'epochs pour chaque perceptron

adb.fit(X_train, y_train)

# Évaluer
y_pred = adb.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print("Test accuracy =", accuracy)

Test accuracy = 0.9277777777777778


In [8]:
Training (M, alpha, epochs):

SyntaxError: invalid syntax (1746772930.py, line 1)

In [10]:
from sklearn.metrics import accuracy_score

# Plusieurs valeurs possibles
M_values = [10, 30, 50, 70]
alpha_values = [0.01, 0.1, 1.0]
epochs_values = [5, 10]

results = []

for M in M_values:
    for alpha in alpha_values:
        for ep in epochs_values:
            adb = MultiClassAdaBoostSAMME(
                base_estimator_class=MultiClassPerceptronOvR,
                n_classes=n_classes, 
                M=M,
                alpha=alpha,
                epochs=ep
            )
            adb.fit(X_train, y_train)
            y_pred = adb.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            results.append((M, alpha, ep, acc))

# Affichage des résultats
for (M, alpha, ep, acc) in results:
    print(f"M={M}, alpha={alpha}, epochs={ep} => Test accuracy = {acc:.4f}")

M=10, alpha=0.01, epochs=5 => Test accuracy = 0.9222
M=10, alpha=0.01, epochs=10 => Test accuracy = 0.9333
M=10, alpha=0.1, epochs=5 => Test accuracy = 0.8917
M=10, alpha=0.1, epochs=10 => Test accuracy = 0.9667
M=10, alpha=1.0, epochs=5 => Test accuracy = 0.9222
M=10, alpha=1.0, epochs=10 => Test accuracy = 0.9306
M=30, alpha=0.01, epochs=5 => Test accuracy = 0.9444
M=30, alpha=0.01, epochs=10 => Test accuracy = 0.9306
M=30, alpha=0.1, epochs=5 => Test accuracy = 0.9306
M=30, alpha=0.1, epochs=10 => Test accuracy = 0.9250
M=30, alpha=1.0, epochs=5 => Test accuracy = 0.9444
M=30, alpha=1.0, epochs=10 => Test accuracy = 0.9417
M=50, alpha=0.01, epochs=5 => Test accuracy = 0.9222
M=50, alpha=0.01, epochs=10 => Test accuracy = 0.9333
M=50, alpha=0.1, epochs=5 => Test accuracy = 0.9278
M=50, alpha=0.1, epochs=10 => Test accuracy = 0.9389
M=50, alpha=1.0, epochs=5 => Test accuracy = 0.9222
M=50, alpha=1.0, epochs=10 => Test accuracy = 0.9528
M=70, alpha=0.01, epochs=5 => Test accuracy = 0.9

In [ ]:
Afficher une matrice de confusion et un rapport de classification

In [12]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = adb.predict(X_test)

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix :")
print(cm)

# Rapport de classification
report = classification_report(y_test, y_pred)
print("Classification Report :")
print(report)

Confusion Matrix :
[[32  0  1  0  0  0  0  0  0  0]
 [ 0 27  0  0  0  0  0  0  0  1]
 [ 0  1 29  2  0  0  0  0  1  0]
 [ 0  0  0 33  0  1  0  0  0  0]
 [ 0  2  0  0 44  0  0  0  0  0]
 [ 0  0  0  0  0 42  1  0  0  4]
 [ 0  0  0  0  0  1 34  0  0  0]
 [ 0  0  0  0  0  0  0 33  0  1]
 [ 0  0  0  0  0  0  0  0 30  0]
 [ 0  2  0  0  0  0  0  0  3 35]]
Classification Report :
              precision    recall  f1-score   support

           0       1.00      0.97      0.98        33
           1       0.84      0.96      0.90        28
           2       0.97      0.88      0.92        33
           3       0.94      0.97      0.96        34
           4       1.00      0.96      0.98        46
           5       0.95      0.89      0.92        47
           6       0.97      0.97      0.97        35
           7       1.00      0.97      0.99        34
           8       0.88      1.00      0.94        30
           9       0.85      0.88      0.86        40

    accuracy                  

In [ ]:
cross-validation

In [14]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_cv = []
for train_index, test_index in skf.split(X, y):
    X_tr, X_te = X[train_index], X[test_index]
    y_tr, y_te = y[train_index], y[test_index]
    
    adb_cv = MultiClassAdaBoostSAMME(
        base_estimator_class=MultiClassPerceptronOvR,
        n_classes=n_classes, 
        M=50,
        alpha=0.1,
        epochs=5
    )
    adb_cv.fit(X_tr, y_tr)
    y_pred_cv = adb_cv.predict(X_te)
    acc_cv = accuracy_score(y_te, y_pred_cv)
    scores_cv.append(acc_cv)

print("Scores de CV :", scores_cv)
print("Moyenne      :", np.mean(scores_cv))
print("Écart-type   :", np.std(scores_cv))

Scores de CV : [0.9361111111111111, 0.9333333333333333, 0.9331476323119777, 0.935933147632312, 0.8941504178272981]
Moyenne      : 0.9265351284432064
Écart-type   : 0.01624027393582752


In [15]:
Confrontation à d’autres classifieurs (référence):


SyntaxError: invalid character '’' (U+2019) (2432232995.py, line 1)

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# AdaBoost classique (sur un arbre de décision, par ex.)
ada_sklearn = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=50
)
ada_sklearn.fit(X_train, y_train)
y_pred_sklearn = ada_sklearn.predict(X_test)
print("Sklearn AdaBoost accuracy =", accuracy_score(y_test, y_pred_sklearn))

Sklearn AdaBoost accuracy = 0.21944444444444444


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
